In [ ]:
RANDOM_SEED = 42

In [ ]:
path_ = "Scalograms-stacked-wavdec"

In [ ]:
import os
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
#Preparing images for model training --- image size = 256 x 256
train_batch = 64
test_batch = 64
train_set= image_dataset_from_directory(path_,
                                labels='inferred',
                                label_mode='binary',
                                batch_size=train_batch,
                                seed=RANDOM_SEED,
                                shuffle=True,
                                validation_split=0.2,
                                subset='training')
                                
val_set =  image_dataset_from_directory(path_,
                                labels='inferred',
                                label_mode='binary',
                                batch_size=test_batch,
                                seed=RANDOM_SEED,
                                shuffle=True,
                                validation_split=0.2,
                                subset='validation')

In [ ]:
test_ds_size = int(int(val_set.__len__())*0.5) # test-test set has 478 images and val set has 1647 images == # 9563 * 0.05
test_test_set = val_set.take(test_ds_size)
test_set = val_set.skip(test_ds_size)

In [ ]:
print(val_set.__len__(), train_set.__len__(), test_set.__len__(), test_test_set.__len__(), sep='\n')

In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
#from keras.optimizers import gradient_descent_v2 
from keras import regularizers, initializers
from livelossplot import PlotLossesKeras
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
inception = InceptionV3(include_top=False, weights='imagenet', input_shape=(256,256,3))
output = inception.layers[-1].output
output = Flatten()(output)
inception = Model(inception.input, output)
for layer in inception.layers:
    layer.trainable = False
inception.summary()

In [ ]:
model_1 = Sequential()

model_1.add(inception)
model_1.add(Dense(64,activation='relu', input_dim=(256,256,3),  kernel_initializer=initializers.HeNormal(), kernel_regularizer=regularizers.L2(0.5)))
model_1.add(Dense(64,activation='relu', kernel_regularizer=regularizers.L2(0.5)))
model_1.add(Dense(1,activation='sigmoid'))

model_1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy','Recall','Precision','AUC'])   

In [ ]:
root_path = "Models/music-scalograms-TL-inception-checkpoints/"
filepath = root_path + 'music-scalograms-TL-l2-5-inception-model-t1.h5'

k=2
if not os.path.exists(root_path): os.makedirs(root_path)
else: 
    while(os.path.exists(filepath)): 
        filepath = filepath[:-4] + str(k) + '.h5' 
        k = k+1

callbacks = [#EarlyStopping(monitor='val_accuracy', patience=4),
            ModelCheckpoint(filepath=filepath, monitor="val_accuracy", mode='max', save_best_only=True),
            PlotLossesKeras()]

In [ ]:
h = model_1.fit(train_set,
                epochs=25,
                #steps_per_epoch=64,
                validation_data=test_set,
                #validation_steps=32,
                callbacks=callbacks
                )

In [ ]:
import pandas as pd

pd.DataFrame.from_dict(h.history).to_csv(filepath[:-2] + 'csv' , index=False)
#data=pd.read_csv("Models/scalogram-base-model-checkpoints/scalogram-base-model-1.csv")

In [ ]:
model_1.evaluate(test_test_set)